In [ ]:
# 광진구, 성동구, 동작구, 관악구, 강동구, 송파구, 강북구, 성북구, 강남구, 서초구, 중구, 강서구
# gu 컬럼에서 위 구 항목에 대응하는 녀석들로 파티션, school_name컬럼에 "초등학교", "중학교", "고등학교"인 애들만 추려서 10개씩
import pandas as pd


school_data = pd.read_csv("school_info.csv")

# List of districts (구) to filter
districts = ["광진구", "성동구", "동작구", "관악구", "강동구", "송파구", 
             "강북구", "성북구", "강남구", "서초구", "중구", "강서구"]

def categorize_school_type(name):
    if "초등학교" in name:
        return "초등학교"
    elif "중학교" in name:
        return "중학교"
    elif "고등학교" in name:
        return "고등학교"
    else:
        return "기타"

school_data['school_type'] = school_data['school_name'].apply(categorize_school_type)
school_data = school_data[school_data['gu'].isin(districts)]
school_data = school_data[school_data["school_type"].isin(['중학교', '고등학교', '초등학교'])]

In [ ]:
len(school_data['school_name'].unique())

In [ ]:
list(school_data['school_name'])

In [ ]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import Select
import sqlite3
import pandas as pd
import os

def write_to_notdone(entry):
    log_filename = 'notdone.txt'
    
    with open(log_filename, 'a') as log_file:
        log_file.write(entry + '\n')

def extract_html(school_name):
    
    ##### 크롤링
    driver = None
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get("https://www.schoolinfo.go.kr/Main.do")
        time.sleep(1)

        input_xpath = "/html/body/div[1]/div[2]/div/div[1]/div/div[2]/form/input[11]"
        input_element = driver.find_element(By.XPATH, input_xpath)
        input_element.send_keys(school_name)
        time.sleep(1)

        search_button_xpath = "/html/body/div[1]/div[2]/div/div[1]/div/div[2]/form/button"
        driver.find_element(By.XPATH, search_button_xpath).click()
        time.sleep(1)

        link_xpath = "/html/body/div[1]/div/div/div[2]/div/div[1]/p/a"
        driver.find_element(By.XPATH, link_xpath).click()
        time.sleep(1)

        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)

        menu_item_xpath = "/html/body/div/div[3]/div[2]/ul/li[2]/a"
        driver.find_element(By.XPATH, menu_item_xpath).click()
        time.sleep(1)

        final_item_xpath = "/html/body/div/div[3]/div[2]/div/div[2]/ul/li[14]/a"
        driver.find_element(By.XPATH, final_item_xpath).click()
        time.sleep(1)

        # 반복적으로 입력을 받고 HTML을 추출하는 부분
        while True:
            input_field_xpath = "/html/body/div/div[3]/div[4]/form/div/div[2]/div/div/table/tbody/tr[4]/td[1]/input"
            submit_button_xpath = "/html/body/div/div[3]/div[4]/form/div/div[2]/div/div/table/tbody/tr[4]/td[2]/button"
            status_message_xpath = "/html/body/div/div[3]/div[4]/form/div/div[2]/div/div/p[2]/b"
            screenshot_element_xpath = "/html/body/div/div[3]/div[4]/form/div/div[2]/div/div/table/tbody/tr[1]/td[1]/div/img"

            input_field = driver.find_element(By.XPATH, input_field_xpath)
            ActionChains(driver).move_to_element(input_field).perform()

            screenshot_element = driver.find_element(By.XPATH, screenshot_element_xpath)
            screenshot_element.screenshot(f'captchas/{school_name}.png')
            
            check_nums = input("Enter the number: ")
            input_field.send_keys(check_nums)
            time.sleep(1)

            driver.find_element(By.XPATH, submit_button_xpath).click()
            time.sleep(1)

            try:
                # 상태 메시지 확인, 다시 입력 받기
                driver.find_element(By.XPATH, status_message_xpath)
                continue
            except:
                file_path = f'captchas/{school_name}.png'
                base, extension = os.path.splitext(file_path)
                new_file_path = f"{base}_{check_nums}{extension}"
                os.rename(file_path, new_file_path)
                break
        
        time.sleep(1)
        html_content = driver.page_source
        html_filename = f"school_violence_htmls/{school_name}_recent_one_year.html"
        with open(html_filename, 'w', encoding='utf-8') as html_file:
            html_file.write(html_content)
        print(f"{html_filename} saved successfully.")
        
    except Exception as e:
        print(f'{school_name} failed.')
        print(e)
        write_to_notdone(f'#####{school_name} --- {e}')
        
    finally:
        driver.quit()


#####
##### 완료된 학교 로그 업데이트
#####

import os

except_list = []

def contains_school(filename):
    for i in range(len(filename) - 1):
        if filename[i:i + 2] == '학교':
            return True
    return False

def write_to_done(entry):
    log_filename = 'done.txt'
    
    with open(log_filename, 'a') as log_file:
        log_file.write(entry + '\n')

# done.txt 만들기
for filename in os.listdir('school_violence_htmls'):
    if contains_school(filename) == True:
        for i in range(len(filename)):
            if filename[i] == '_':
                write_to_done(filename[:i])
                break

# 제외 리스트 만들기
with open('done.txt', 'r') as file:
    done_list = [line.strip() for line in file]
    except_list += list(set(done_list))

with open('notdone.txt', 'r') as file:
    for line in file:
        school_name = ''
        if line[0] == '#':
            school_name = line.split('---')[0][5:]
            school_name = school_name.strip()
            except_list.append(school_name)

print(f'except_list: {except_list}')

for school_name in list(set(list(school_data['school_name']))):
    if not school_name in except_list:
        extract_html(school_name)          

